# **Artificial Neural Networks and Deep Learning**

---

## Challenge 2

# 🐉 **Ensemble**

In [1]:
# Download data
import gdown

gdown.download(id = "1Q6QQe2NGzfELjPprDsVd2A_PKSG-UYAE", output = "vgg19_ranger.csv",         quiet = False)
gdown.download(id = "1zcJW5CjRX1yWbtGJ6HzGHqwDwFIVHTua", output = "vgg16_ranger.csv",         quiet = False)
gdown.download(id = "1n6qr9Qw7HrKz55PB43V7PRdIUtJ96NHD", output = "vgg16_adam.csv",           quiet = False)
gdown.download(id = "1sRE9yqcnJg9UYemRPQ_-aVW3A00BchWW", output = "resnet50_ranger.csv",      quiet = False)
gdown.download(id = "1kunUO3fxqgK83W9HVTZ1lqv8WJ_P9rjB", output = "vgg16_adam_synthetic.csv", quiet = False)
gdown.download(id = "1uAniu954LbGQSlYMfJpE214mG-E3fsh5", output = "inceptionv3_ranger.csv",   quiet = False)

Downloading...
From: https://drive.google.com/uc?id=1Q6QQe2NGzfELjPprDsVd2A_PKSG-UYAE
To: /content/vgg19_ranger.csv
100%|██████████| 10.8k/10.8k [00:00<00:00, 3.01MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zcJW5CjRX1yWbtGJ6HzGHqwDwFIVHTua
To: /content/vgg16_ranger.csv
100%|██████████| 10.9k/10.9k [00:00<00:00, 28.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1n6qr9Qw7HrKz55PB43V7PRdIUtJ96NHD
To: /content/vgg16_adam.csv
100%|██████████| 10.9k/10.9k [00:00<00:00, 11.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1sRE9yqcnJg9UYemRPQ_-aVW3A00BchWW
To: /content/resnet50_ranger.csv
100%|██████████| 11.1k/11.1k [00:00<00:00, 13.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kunUO3fxqgK83W9HVTZ1lqv8WJ_P9rjB
To: /content/vgg16_adam_synthetic.csv
100%|██████████| 11.0k/11.0k [00:00<00:00, 31.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uAniu954LbGQSlYMfJpE214mG-E3fsh5
To: /content/inceptionv3_ranger.csv
100%|██████████| 10.9k/1

'inceptionv3_ranger.csv'

In [2]:
import pandas as pd

# Load all 6 files
s1 = pd.read_csv("vgg19_ranger.csv")
s2 = pd.read_csv("vgg16_ranger.csv")
s3 = pd.read_csv("vgg16_adam.csv")
s4 = pd.read_csv("resnet50_ranger.csv")
s5 = pd.read_csv("vgg16_adam_synthetic.csv")
s6 = pd.read_csv("inceptionv3_ranger.csv")

# Rename label columns so we know who is who
s1 = s1.rename(columns={"label": "label_1"})
s2 = s2.rename(columns={"label": "label_2"})
s3 = s3.rename(columns={"label": "label_3"})
s4 = s4.rename(columns={"label": "label_4"})
s5 = s5.rename(columns={"label": "label_5"})
s6 = s6.rename(columns={"label": "label_6"})

# Merge on sample_index
merged = (
    s1.merge(s2, on="sample_index")
      .merge(s3, on="sample_index")
      .merge(s4, on="sample_index")
      .merge(s5, on="sample_index")
      .merge(s6, on="sample_index")
)

print("Merged shape:", merged.shape)
print(merged.head())

# Define vgg models as high priority
PRIORITY_MODELS = ["label_1", "label_2", "label_3"]

from collections import Counter

# Break eventual ties
def majority_vote(row):
    labels = [
        row["label_1"],
        row["label_2"],
        row["label_3"],
        row["label_4"],
        row["label_5"],
        row["label_6"],
    ]

    counts = Counter(labels)
    most_common = counts.most_common()

    # Highest vote count
    max_votes = most_common[0][1]

    # All labels that have max_votes
    tied_labels = [lbl for lbl, cnt in most_common if cnt == max_votes]

    # Case 1: no tie → just return winner
    if len(tied_labels) == 1:
        return tied_labels[0]

    # Case 2: tie → check preferred models (vgg models)
    priority_votes = [row[m] for m in PRIORITY_MODELS]
    for lbl in priority_votes:
        if lbl in tied_labels:
            return lbl  # first matching preferred label wins

    # Case 3: still tie → fallback: pick first tied label (deterministic)
    return tied_labels[0]

# Apply ensemble
merged["label"] = merged.apply(majority_vote, axis=1)

# Build final submission
submission = merged[["sample_index", "label"]].copy()
submission = submission.sort_values("sample_index")

print("Submission shape:", submission.shape)
print(submission.head())

# Save & download
from google.colab import files

out_name = "test_labels_ensemble_final.csv"
submission.to_csv(out_name, index = False)
files.download(out_name)

Merged shape: (477, 7)
   sample_index    label_1    label_2    label_3          label_4    label_5  \
0  img_0000.png  Luminal A  Luminal B  Luminal B        Luminal A  Luminal A   
1  img_0001.png    HER2(+)  Luminal B  Luminal B        Luminal B  Luminal B   
2  img_0002.png  Luminal A    HER2(+)  Luminal B  Triple negative    HER2(+)   
3  img_0003.png  Luminal A  Luminal A  Luminal B        Luminal B  Luminal B   
4  img_0004.png  Luminal B  Luminal B  Luminal B        Luminal B  Luminal A   

     label_6  
0  Luminal B  
1  Luminal B  
2  Luminal B  
3  Luminal B  
4  Luminal A  
Submission shape: (477, 2)
   sample_index      label
0  img_0000.png  Luminal A
1  img_0001.png  Luminal B
2  img_0002.png    HER2(+)
3  img_0003.png  Luminal B
4  img_0004.png  Luminal B


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>